In [1]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from gcForest import gcForestRegressor as gcfR

In [2]:
train = pd.read_csv('processed_train_fillna_sex.csv')
test = pd.read_csv('processed_test_fillna_sex.csv')

In [3]:
train_y = train.resale_price
train_x = train.drop("resale_price", axis=1)

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

x_scaler = StandardScaler()
train_x = x_scaler.fit_transform(train_x)
test = x_scaler.transform(test)

y_scaler = StandardScaler()
train_y = y_scaler.fit_transform(np.array(train_y).reshape(-1,1))

In [5]:
#baseline
model = xgboost.XGBRegressor(n_estimators=1000, max_depth=10, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model_gcforest = gcfR.GCForestRegressor(shape_1X=43, window=41, cv_method='sequence', scoring='explained_variance', mgs_criterion='poisson',cascade_criterion="poisson")

In [6]:
#model = xgboost.XGBRegressor(n_estimators=1500, max_depth=15, eta=0.05, subsample=0.8, colsample_bytree=0.8)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_x,train_y,test_size=0.1)

In [8]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, ...)

In [9]:
from sklearn.metrics import mean_absolute_error
t_y = model.predict(X_train)
t_y = y_scaler.inverse_transform(t_y.reshape(-1,1))
y_train = y_scaler.inverse_transform(y_train.reshape(-1,1))
train_error = mean_absolute_error(np.array(y_train).reshape(-1),np.array(t_y).reshape(-1))


p_y = model.predict(X_val)
p_y = y_scaler.inverse_transform(p_y.reshape(-1,1))
y_val = y_scaler.inverse_transform(y_val.reshape(-1,1))
val_error = mean_absolute_error(np.array(y_val).reshape(-1),np.array(p_y).reshape(-1))

In [10]:
print('train_error: ',train_error)
print('val_error: ',val_error)

train_error:  7647.369379574046
val_error:  11410.863883871805


In [13]:
model.fit(train_x, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, ...)

In [14]:
predict_y = model.predict(test)
predict_y = y_scaler.inverse_transform(predict_y.reshape(-1,1))

In [15]:
result = pd.DataFrame()
result.insert(result.shape[0], 'Id', range(len(predict_y)))
result.insert(1, 'Predicted', predict_y)
result.to_csv('result_xgboost_s1.csv', index = 0)